### Personal Notes
* Timedeltas
* Seaborn
* Longtitude, latitude plots

# Introduction
In 2016, Cyclistic launched a successful bike-share offering. Since then, the program has grown to a fleet of 5,824 bicycles that are geotracked and locked into a network of 692 stations across Chicago. The bikes can be unlocked from one station and returned to any other station in the system anytime.

Until now, Cyclistic’s marketing strategy relied on building general awareness and appealing to broad consumer segments. One approach that helped make these things possible was the flexibility of its pricing plans: single-ride passes, full-day passes, and annual memberships. Customers who purchase single-ride or full-day passes are referred to as casual riders. Customers who purchase annual memberships are Cyclistic members.

# Stage 1: Questions and Goals
#### Stakeholders:  
 **Lily Moreno** : The director of marketing and your manager. Moreno is responsible for the development of campaigns and initiatives to promote the bike-share program. These may include email, social media, and other channels.  
 **Cyclistic marketing analytics team** : A team of data analysts who are responsible for collecting, analyzing, and reporting data that helps guide Cyclistic marketing strategy. You joined this team six months ago and have been busy learning about Cyclistic’s mission and business goals — as well as how you, as a junior data analyst, can help Cyclistic achieve them.  
 **Cyclistic executive team** : The notoriously detail-oriented executive team will decide whether to approve the recommended marketing program.  
#### Questions:
1. How annual members and casual riders use Cyclistic bikes differently?
2. Why would casual riders buy Cyclistic annual membership?
3. How can Cyclistic use digital media to influence casual riders to become members?

# Stage 2: Prepare
**Data**: [click here](https://divvy-tripdata.s3.amazonaws.com/index.html)  
**Licence**: [click here](https://www.divvybikes.com/data-license-agreement)  
**Description**: The data contains raw information about the customers using Cyclistic services in 2019.  
**Problems**:  
* Some columns contain incorrect format
* Some columns contain empty cells  

All the problems will be handled during cleaning and the description will be provided in the stage 3.

# Stage 3: Cleaning
### 3.1 Uploading and processing the data
During this case study the common libraries will be used, namely `pandas` (working with data frames), `numpy` (data structures and math operations), `matplotlib` and `seaborn` (both for data visualization)

In [46]:
# Importing libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime

# Creating data frames
df1 = pd.read_csv('~/Downloads//cyclistic/202101-divvy-tripdata.csv')
df2 = pd.read_csv('~/Downloads//cyclistic/202102-divvy-tripdata.csv')
df3 = pd.read_csv('~/Downloads//cyclistic/202103-divvy-tripdata.csv')
df4 = pd.read_csv('~/Downloads//cyclistic/202104-divvy-tripdata.csv')
df5 = pd.read_csv('~/Downloads//cyclistic/202105-divvy-tripdata.csv')
df6 = pd.read_csv('~/Downloads//cyclistic/202106-divvy-tripdata.csv')
df7 = pd.read_csv('~/Downloads//cyclistic/202107-divvy-tripdata.csv')
df8 = pd.read_csv('~/Downloads//cyclistic/202108-divvy-tripdata.csv')

# Merging data frames (SQL-like union)
df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8])
df.reset_index(drop=True, inplace=True)
df.head(4)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,E19E6F1B8D4C42ED,electric_bike,2021-01-23 16:14:19,2021-01-23 16:24:44,California Ave & Cortez St,17660,NaN,NaN,41.900341,-87.696743,41.89,-87.72,member
1,DC88F20C2C55F27F,electric_bike,2021-01-27 18:43:08,2021-01-27 18:47:12,California Ave & Cortez St,17660,NaN,NaN,41.900333,-87.696707,41.90,-87.69,member
2,EC45C94683FE3F27,electric_bike,2021-01-21 22:35:54,2021-01-21 22:37:14,California Ave & Cortez St,17660,NaN,NaN,41.900313,-87.696643,41.90,-87.70,member
3,4FA453A75AE377DB,electric_bike,2021-01-07 13:31:13,2021-01-07 13:42:55,California Ave & Cortez St,17660,NaN,NaN,41.900399,-87.696662,41.92,-87.69,member


### 3.2 Handling empty values
The table below shows that the data frame contains cells in `start_station_name`, `start_station_id`, `end_station_name`, `end_station_id`, `end_lat`, and `end_lng` columns. Since I am not able to find the information about latitude and lontitude, the rows will be deleted. The information in the rest of the columns is not crucial, so the empty cells will be replaced with `0` and `Unknown` values.

In [47]:
# Visualization of empty rows
df.isna().sum()

ride_id                    0
rideable_type              0
started_at                 0
ended_at                   0
start_station_name    363133
start_station_id      363132
end_station_name      392390
end_station_id        392390
start_lat                  0
start_lng                  0
end_lat                 3357
end_lng                 3357
member_casual              0
dtype: int64

In [48]:
 # Filling the empty values
for i in df:
    if i in ['start_station_name', 'end_station_name']:
        df[i] = df[i].fillna('Unknown')
    elif i in ['start_station_id', 'end_station_id']:
        df[i] = df[i].fillna(0)
    elif i in ['end_lat', 'end_lng']:
        df.dropna(subset=['end_lat', 'end_lng'], inplace=True)

# Checking empty values after cleaning        
df.isna().sum()

ride_id               0
rideable_type         0
started_at            0
ended_at              0
start_station_name    0
start_station_id      0
end_station_name      0
end_station_id        0
start_lat             0
start_lng             0
end_lat               0
end_lng               0
member_casual         0
dtype: int64

### 3.3 Incorect data types
Columns `started_at` and `ended_at` represent timestamp during which the service was provided, however these columns have data type `string`. I am going to convert them to `datetime64` type.

In [49]:
# Existing data types
df.dtypes

ride_id                object
rideable_type          object
started_at             object
ended_at               object
start_station_name     object
start_station_id       object
end_station_name       object
end_station_id         object
start_lat             float64
start_lng             float64
end_lat               float64
end_lng               float64
member_casual          object
dtype: object

In [50]:
# Changing the data types
for i in df:
    if i in ['started_at', 'ended_at']:
        df[i] = pd.to_datetime(df[i])

# Checking data types after cleaning
df.dtypes

ride_id                       object
rideable_type                 object
started_at            datetime64[ns]
ended_at              datetime64[ns]
start_station_name            object
start_station_id              object
end_station_name              object
end_station_id                object
start_lat                    float64
start_lng                    float64
end_lat                      float64
end_lng                      float64
member_casual                 object
dtype: object

In [ ]:
# Total number of rows
df.count()[0]

# Stage 4: Analysis

1. How annual members and casual riders use Cyclistic bikes differently?
2. Why would casual riders buy Cyclistic annual membership?
3. How can Cyclistic use digital media to influence casual riders to become members?

* Bike type usage by usercount
* Mean travel time by user
* Mean travel distance by user
* Travel by day of week member/casual

### 4.1 Additional calculations
Following, I am going to calculate parameters that is not in the data frame, but will be very useful during the analysis process.  

Parameter `duration_minutes` is calculated by subtracting time when a ride was ended and starting time:

In [ ]:
# Inserting a new column showing duration of a ride in minutes
duration = df['ended_at'] - df['started_at']
df.insert(4, 'duration_minutes', np.round(duration.dt.seconds/60, 1))

The new column `start_date` will be useful to represent changing overtime. Basically, I am going to change the format of the `started_at` parameter to a standart year-month-day type:

In [ ]:
# Inserting a new column showing a start date
start_date = pd.to_datetime(df['started_at']).dt.date
df.insert(2, 'start_date', start_date)
df['start_date'] = pd.to_datetime(df['start_date'])

Having latitude and longitude parameters allows to calculate the distance between starting point and destination by creating a custom function.
The result of the function will be represented in kilometers and can be checked [here](https://www.meridianoutpost.com/resources/etools/calculators/calculator-latitude-longitude-distance.php?).

In [ ]:
# Create a function to calculate the distance between two points
def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    All args must be of equal length.    
    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    
    return km

# Add new column with the distance between points
distance_km = haversine_np(df['start_lng'], df['start_lat'], df['end_lng'], df['end_lat'])
df.insert(10, 'distance_km', distance_km)

Identifying a day of a week according to `start_date` can be useful when visualizing the distribution of the rides between week days:

In [ ]:
# Add weekday column
day_of_week = df['start_date'].dt.day_name()
df.insert(3, 'day_of_week', df['start_date'].dt.day_name())

Quick check the updated data frame and its data types:

In [ ]:
# Quick check all new columns have been added
df.sample(5)

In [ ]:
# Quick check new columns
df.dtypes

### 4.2. Members vs. Casual
In the previous section, we determined that the number of rides between Jan - Aug 2021 is `3596815`. This number is destributed almost 50/50 between casual drivers and drivers with the subscription.

In [ ]:
# Data frame grouped by driver type (count)
member = df.groupby('member_casual').count().reset_index(drop=False)[['member_casual', 'ride_id']]

# Parameters
values = [i for i in member['ride_id']]
labels = [j for j in member['member_casual']]
colors = sns.color_palette('pastel')[0:2]

# Pie chart
plt.figure(figsize=(8, 8))
plt.pie(values, labels=labels, colors=colors, autopct = '%.1f %%', explode=[0.01, 0.01], textprops={'fontsize': 15})

# Non-data parameters
plt.title('Members vs. Casual Drivers', fontdict={'fontweight':'bold', 'fontsize':20})

plt.show()

### 4.2 Daily distance casual vs. member

In [ ]:
member_distance = df[['start_date', 'member_casual', 'distance_km']]
member_distance = member_distance.groupby(['start_date', 'member_casual']).sum()  # sum or mean?
member_distance.reset_index(drop=False, inplace=True)

plt.figure(figsize=(17, 9))

sns.lineplot(data=member_distance, x='start_date', y='distance_km', hue='member_casual')

plt.show()

### 4.3 Casual/Member average duration

In [ ]:
# Casual/Member avg DURATION and bike type
# Subplots Faced grid: https://seaborn.pydata.org/generated/seaborn.FacetGrid.html
# Simple: https://www.python-graph-gallery.com/grouped-barplot
# Faced Grid: https://seaborn.pydata.org/tutorial/axis_grids.html

avg_duration = df[['member_casual', 'rideable_type', 'duration_minutes']].groupby(['member_casual', 'rideable_type']).mean()
avg_duration.reset_index(drop=False, inplace=True)

g = sns.FacetGrid(avg_duration, col='member_casual')
g.map(sns.barplot, 'rideable_type', 'duration_minutes', order=['classic_bike', 'docked_bike', 'electric_bike'], palette='magma')

# Non-data parameters
g.fig.set_size_inches(10,10)
g.set_titles(col_template="{col_name}")
g.set_axis_labels('TYPE', 'AVG MINUTES')
g.set_xticklabels(['a', 'b', 'c'])

plt.show()

### 4.4 Casual/Member average distance

In [ ]:
# Casual/Member avg DISTANCE and bike type
avg_distance = df[['member_casual', 'rideable_type', 'distance_km']].groupby(['member_casual', 'rideable_type']).mean()
avg_distance.reset_index(drop=False, inplace=True)

g = sns.FacetGrid(avg_distance, col='member_casual')
g.map(sns.barplot, 'rideable_type', 'distance_km', order=['classic_bike', 'docked_bike', 'electric_bike'], palette='mako')

# Non-data parameters
g.fig.set_size_inches(10,10)
g.set_titles(col_template="{col_name}")
g.set_axis_labels('TYPE', 'AVG KILOMETERS')
g.set_xticklabels(['a', 'b', 'c'])

plt.show()

In [ ]:
# Casual COUNT
df.loc[df['member_casual'] == 'casual', ['rideable_type', 'duration_minutes', 'distance_km']].groupby('rideable_type').count()

### Day of week